# Lesson 03: Image Cropping

Cropping extracts a **rectangular region of interest (ROI)** from an image.  
In OpenCV / NumPy, images are 2D arrays, so cropping is just **array slicing**.

## Coordinate system:
```
(0,0) ──────────── x (width)
  │
  │   img[y1:y2, x1:x2]
  │
  y (height)
```

**Note:** rows = y-axis = height; columns = x-axis = width

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img_bgr = cv2.imread('sealion_hero.png')
img     = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
h, w    = img.shape[:2]

print(f'Image size: {w} × {h}')

## 1. Basic Cropping — NumPy Array Slicing

Syntax: `img[y_start:y_end, x_start:x_end]`

In [ ]:
# Crop top-left quarter
top_left     = img[0:h//2, 0:w//2]

# Crop center region
center       = img[h//4:3*h//4, w//4:3*w//4]

# Crop bottom-right quarter
bottom_right = img[h//2:, w//2:]

# Crop a strip — top 20% of image
top_strip    = img[0:int(h*0.2), :]

fig, axes = plt.subplots(1, 4, figsize=(16, 4))
for ax, (im, t) in zip(axes, [
    (top_left,     f'Top-left ({top_left.shape[1]}×{top_left.shape[0]})'),
    (center,       f'Center crop'),
    (bottom_right, f'Bottom-right'),
    (top_strip,    f'Top strip (20%)'),
]):
    ax.imshow(im)
    ax.set_title(t)
    ax.axis('off')
plt.tight_layout()
plt.show()

## 2. Cropping by Specifying a ROI

A more structured approach: define the ROI as `(x, y, width, height)`.

In [ ]:
def crop_roi(image, x, y, roi_w, roi_h):
    """Crop a region starting at (x,y) with size roi_w×roi_h."""
    return image[y:y+roi_h, x:x+roi_w]

# Define ROI parameters
roi_x, roi_y = w//4, h//4
roi_w, roi_h = w//2, h//2

cropped = crop_roi(img, roi_x, roi_y, roi_w, roi_h)

# Draw the ROI rectangle on the original for visualization
vis = img.copy()
cv2.rectangle(vis, (roi_x, roi_y), (roi_x+roi_w, roi_y+roi_h), (255, 0, 0), 4)

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
axes[0].imshow(vis);     axes[0].set_title('Original with ROI box'); axes[0].axis('off')
axes[1].imshow(cropped); axes[1].set_title(f'Cropped region ({roi_w}×{roi_h})'); axes[1].axis('off')
plt.tight_layout()
plt.show()

## 3. Cropping — Shared Memory Warning!

NumPy slicing returns a **view** (not a copy).  
Modifying the crop modifies the original too — use `.copy()` to avoid this.

In [ ]:
original = img.copy()

# View (shared memory) — modifying will affect original!
crop_view = original[50:200, 50:300]
crop_view[:] = [255, 0, 0]  # fill with red

# Copy (independent) — modifications won't affect original
original2 = img.copy()
crop_copy = original2[50:200, 50:300].copy()
crop_copy[:] = [0, 0, 255]  # fill with blue — original2 is unchanged

fig, axes = plt.subplots(1, 3, figsize=(13, 4))
axes[0].imshow(img);       axes[0].set_title('img (not modified)'); axes[0].axis('off')
axes[1].imshow(original);  axes[1].set_title('original AFTER crop_view[:] = red\n(view modification!)'); axes[1].axis('off')
axes[2].imshow(original2); axes[2].set_title('original2 AFTER crop_copy[:] = blue\n(copy — unaffected)'); axes[2].axis('off')
plt.tight_layout()
plt.show()

## 4. Resizing a Crop to a Standard Size

Often you crop and then resize to a fixed size for further processing.

In [ ]:
# Crop different areas
crops = [
    img[0:h//3, 0:w//3],
    img[h//3:2*h//3, w//3:2*w//3],
    img[2*h//3:, 2*w//3:],
]

# Resize all to same target size
TARGET = (200, 200)
resized_crops = [cv2.resize(c, TARGET) for c in crops]

fig, axes = plt.subplots(2, 3, figsize=(12, 7))
for i, (crop, res) in enumerate(zip(crops, resized_crops)):
    axes[0, i].imshow(crop); axes[0, i].set_title(f'Crop {i+1} ({crop.shape[1]}×{crop.shape[0]})'); axes[0, i].axis('off')
    axes[1, i].imshow(res);  axes[1, i].set_title(f'Resized to {TARGET[0]}×{TARGET[1]}'); axes[1, i].axis('off')
plt.tight_layout()
plt.show()

## Summary

- Cropping = NumPy slicing: `img[y1:y2, x1:x2]`
- Coordinate order: **rows (y) first, then columns (x)**
- Use `:` to take the full range of an axis
- **Always use `.copy()`** if you plan to modify the crop independently of the original
- Crop then `cv2.resize()` when you need a fixed output size